<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 5 AutoGluon</h4>
</center>

In [2]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [20]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product.csv", header = None)
df.columns = ["timestamp", "item_id", "target"]
df.head(5)

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [21]:
df_product = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_products.csv", header = None)
df_product.columns = ["item_id", "cat1", "cat2", "cat3", "brand", "sku_size"]
df_product.head(5)

,item_id,cat1,cat2,cat3,brand,sku_size
0,20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
1,20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
2,20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
3,20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
4,20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [22]:
df_related = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_related_product.csv", header = None)
df_related.columns = ["timestamp", "item_id", "month", "quarter", "days_in_month", "sundays", "saturdays", "tn_m3", "tn_m6", "tn_m12"]
df_related.head(5)

,timestamp,item_id,month,quarter,days_in_month,sundays,saturdays,tn_m3,tn_m6,tn_m12
0,2017-01-01,20001,1,1,31,5,4,NaN,NaN,NaN
1,2017-02-01,20001,2,1,28,4,4,NaN,NaN,NaN
2,2017-03-01,20001,3,1,31,4,4,NaN,NaN,NaN
3,2017-04-01,20001,4,2,30,5,5,934.77222,NaN,NaN
4,2017-05-01,20001,5,2,31,4,4,798.01620,NaN,NaN


In [23]:
df_extendido = pd.merge(df, df_related, how = "left", on = ["timestamp", "item_id"])
print(df_extendido.timestamp.max())
df_extendido.head(5)

2019-12-01


,timestamp,item_id,target,month,quarter,days_in_month,sundays,saturdays,tn_m3,tn_m6,tn_m12
0,2017-01-01,20001,934.77222,1,1,31,5,4,NaN,NaN,NaN
1,2017-01-01,20002,550.15707,1,1,31,5,4,NaN,NaN,NaN
2,2017-01-01,20003,1063.45835,1,1,31,5,4,NaN,NaN,NaN
3,2017-01-01,20004,555.91614,1,1,31,5,4,NaN,NaN,NaN
4,2017-01-01,20005,494.27011,1,1,31,5,4,NaN,NaN,NaN


In [24]:
data = TimeSeriesDataFrame.from_data_frame(df_extendido,
                                           id_column="item_id",
                                           timestamp_column="timestamp",
                                           static_features_df=df_product)
data.head()

,,target,month,quarter,days_in_month,sundays,saturdays,tn_m3,tn_m6,tn_m12
item_id,timestamp,,,,,,,,,
20001,2017-01-01,934.77222,1,1,31,5,4,NaN,NaN,NaN
20002,2017-01-01,550.15707,1,1,31,5,4,NaN,NaN,NaN
20003,2017-01-01,1063.45835,1,1,31,5,4,NaN,NaN,NaN
20004,2017-01-01,555.91614,1,1,31,5,4,NaN,NaN,NaN
20005,2017-01-01,494.27011,1,1,31,5,4,NaN,NaN,NaN


In [25]:
data.static_features.head()

,cat1,cat2,cat3,brand,sku_size
item_id,,,,,
20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [26]:
covariates = data.drop(columns = "target").columns

predictor = TimeSeriesPredictor(target='target',
                                prediction_length=2, 
                                freq="M",
                                eval_metric = "WQL",
                                known_covariates_names = covariates).fit(data, 
                                                                         num_val_windows=3,
                                                                         val_step_size=1)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240620_210525'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       1.83 GB / 7.88 GB (23.2%)
Disk Space Avail:   0.38 GB / 110.21 GB (0.3%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': ['month',
                            'quarter',
                            'days_in_month',
                            'sundays',
                            'saturdays',
                            'tn_m3',
                            'tn_m6',
                            'tn_m12'],
 'num_val_windows': 3

In [28]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.175385,90.418178,5.780694,12
1,AutoETS,-0.194559,32.809046,142.311281,7
2,PatchTST,-0.194902,0.652241,358.118903,11
3,TemporalFusionTransformer,-0.196215,1.066417,1931.135740,9
4,DynamicOptimizedTheta,-0.201713,3.027356,76.703047,6
5,CrostonSBA,-0.209206,1.304995,34.591191,4
6,DeepAR,-0.212285,1.945869,567.828721,10
7,RecursiveTabular,-0.227895,0.349171,133.746740,2
8,SeasonalNaive,-0.230813,0.717928,5.632488,1
9,AutoARIMA,-0.236277,51.367472,173.387783,8


In [29]:
predictor.feature_importance()

Computing feature importance


,importance,stdev,n,p99_low,p99_high
cat1,-0.000308,0.000765,5.0,-0.001882,0.001266
cat2,0.000396,0.001266,5.0,-0.002210,0.003002
cat3,0.000221,0.001067,5.0,-0.001976,0.002419
brand,0.000627,0.000931,5.0,-0.001291,0.002545
sku_size,0.000558,0.001357,5.0,-0.002235,0.003352
month,0.000969,0.002802,5.0,-0.004801,0.006738
quarter,-0.000660,0.001444,5.0,-0.003634,0.002313
days_in_month,0.006413,0.005013,5.0,-0.003910,0.016735
sundays,0.002471,0.001124,5.0,0.000156,0.004787
saturdays,-0.000058,0.001080,5.0,-0.002281,0.002165


In [34]:
known_covariates_future = df_related[df_related.timestamp > df.timestamp.max()]
# Llevamos las fechas futuras a fin de mes porque así lo espera el metodo predict
known_covariates_future['timestamp'] = pd.to_datetime(known_covariates_future['timestamp'])
known_covariates_future['timestamp'] = known_covariates_future['timestamp'] + pd.offsets.MonthEnd(0)
known_covariates_future['timestamp'] = known_covariates_future['timestamp'].dt.strftime('%Y-%m-%d')
print(known_covariates_future.timestamp.min(), known_covariates_future.timestamp.max())
known_covariates_future = TimeSeriesDataFrame.from_data_frame(known_covariates_future,
                                                              id_column="item_id",
                                                              timestamp_column="timestamp")
known_covariates_future.head()

2020-01-31 2020-02-29


month  quarter  days_in_month  sundays  saturdays  \
item_id timestamp                                                       
20001   2020-01-31      1        1             31        4          4   
        2020-02-29      2        1             29        4          5   
20002   2020-01-31      1        1             31        4          4   
        2020-02-29      2        1             29        4          5   
20003   2020-01-31      1        1             31        4          4   

                         tn_m3       tn_m6      tn_m12  
item_id timestamp                                       
20001   2020-01-31  1561.50552  1678.99318  1275.77351  
        2020-02-29  1397.37231  1261.34529  1259.09363  
20002   2020-01-31  1979.53635  1066.44999  1266.78751  
        2020-02-29  1423.57739   813.78215  1043.01349  
20003   2020-01-31  1081.36645   715.20314   964.76919

In [35]:
predictions = predictor.predict(data,
                                known_covariates = known_covariates_future)

data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [36]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1324.417369
3,20002,1139.473463
5,20003,729.469538
7,20004,612.434604
9,20005,606.390113


In [37]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_static_features_and_covariates.csv", sep = ",", index = False)